Comparar KMeans "puro" com KMeans com Improved Density Canopy

Métricas erro Quadrático médio

Quando bancos de dados UCI usar?

1. Calcular MeanDis de cada elemento
2. Calcular densidade de cada elemento
3. O com maior densidade é o centro e eliminar os que não estão dentro do raio.

In [1]:
import pandas as pd
import numpy as np
from time import time

from IPython.display import display

from sklearn.cluster import KMeans

In [2]:
from sklearn.metrics.pairwise import pairwise_distances

# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X, T1, T2, distance_metric='euclidean', filemap=None):
    canopies = dict()
    X1_dist = pairwise_distances(X, metric=distance_metric)
    canopy_points = set(range(X.shape[0]))
    while canopy_points:
        point = canopy_points.pop()
        i = len(canopies)
        canopies[i] = {"c":point, "points": list(np.where(X1_dist[point] < T2)[0])}
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return canopies

In [3]:
def euclideanDistance(vector1, vector2):
        #print(vector1)
        #print(vector2)
        return np.sqrt(np.sum(np.power(vector1-vector2, 2)))

def getDistance(row_center, row_sample):
        #print(row_center)
        row_center = np.asarray(row_center)
        #row_center = np.asarray(row_sample)
        return euclideanDistance(row_center, row_sample)

def getSquaredError(data, kmeans):
    distances = []
    for i in range(k): # Qtd de clusters
        distance = 0
        for index_labels, value_labels in enumerate(kmeans.labels_): #kmeans.labels_ possui a que cluster cada elemento pertence
            if (i == value_labels):
                #print(value_labels)
                distance = distance + getDistance(kmeans.cluster_centers_[value_labels], data.loc[index_labels].values)
        
        distances.append(distance) #Erro quadratico medio de cada cluster
    
    distances = np.asarray(distances)
    error = np.sum(distances)
    return error

In [4]:
# -------------- Density Canopy -------------- #

def MeanDis(D):
    #OBS enumerate com numpy mto mais rápido que iterrows
    n = D.shape[0]
    D = D.values
    meanDis = 0
    
    for i, row_i in enumerate(D):
        for j, row_j in enumerate(D[i+1:,]):
            
            meanDis = meanDis + euclideanDistance(row_i, row_j)
            
    meanDis = (2/(n*(n-1))) * meanDis
    
    return meanDis
def getDensitys(D): #Definition 2
    
    meanDis = MeanDis(D)
    print ("Distância média: ", meanDis)
    #df = pd.DataFrame(np.nan, index=range(D.shape[0]), columns=['density']) #Criando DataFrame vazio
    densitys = np.zeros(D.shape[0], dtype=int)
    
    #display(df.tail())
    auxD = D.values
    
    for i, row_i in enumerate(auxD):
        aux = 0
        for j, row_j in enumerate(auxD):
            if euclideanDistance(row_i, row_j) - meanDis < 0:
                aux = aux + 1
        densitys[i] = aux
    df = pd.DataFrame(data= densitys, columns=["density"])
    #display(df.head(20))
    return df, meanDis

def getAs(auxD, new_densitys):  #Definition 3 NAO ESTÁ CLARO NO ARTIGO
    a = []
    densitys = new_densitys.copy().values
    for i, row_i in enumerate(auxD.values): # confirmar o valor do somatório
        aux = 0
        for j, row_j in enumerate(auxD.values[i+1:] ):
            aux = aux + euclideanDistance(row_i, row_j)
        
        aux = (2/(densitys[i] * (densitys[i] -1))) * aux # calculo de a(i)
        a.append(aux)
    return a

def getSs(auxD, new_densitys):  #Definition 4
    s = []
    densitys = new_densitys.values
    flag = 0 #Se flag =0 entao min dist, se flag =1 retornar max dist
    
    
    for i, row_i in enumerate(auxD.values):
        maxDist = 0
        minDist = float("inf")
        dist = 0
        
        for j, row_j in enumerate(auxD.values):
            
            dist = euclideanDistance(row_i, row_j)
            
            if (dist > maxDist):
                maxDist = dist
            if (dist < minDist):
                minDist = dist
                
                
            if densitys[j] == np.amax(densitys): #Se a densidade  for a maxima quer dizer que p(j) == p(i)
                flag = 1
        
        if flag == 1:
            s.append(maxDist)
        else: # p(j) > p(i)
            s.append(minDist)
    print (s)
    return s
                #print(np.amax(densitys))
        
def getCenter(D): 
    auxD = D.copy()
    df, meanDis = getDensitys(D)
    
    row_i = D.iloc[df.idxmax(axis = 0)].values
    cluster = []
    
    for j, row_j in enumerate(auxD.values):
        if euclideanDistance(row_i, row_j) - meanDis < 0:
            cluster.append(j)
            
    #print (cluster)
    
    #auxD.drop(cluster, inplace=True)
    #display(auxD)
    return row_i, cluster # Elemento central e cluster

def getWeight(D):
    center, cluster = getCenter(D)
    auxD = D.copy()
    
    auxD.drop(cluster, inplace=True) #removendo cluster ja identificado
    auxD.reset_index(inplace=True)
    
    new_densitys, new_meanDis = getDensitys(auxD)
    
    a = getAs(auxD, new_densitys)
    
    s = getSs(auxD, new_densitys)
    
    densitys = new_densitys.values
    aux = 0
    aux_index = None
    for i, row_i in enumerate(auxD.values):
         if (densitys[i] * (1/a[i]) * s[i]) > aux:
                aux = densitys[i] * (1/a[i]) * s[i]
                print("densidade: ",densitys[i], "a : ", a[i], "s: ",s[i])
                aux_index = i
    #print(aux_index)
    #print(auxD.shape)
        
    #print(a)
    #display(new_densitys)
    #display(auxD.head(20))

In [5]:
files = ["soybean-small", "iris", "wine",  "segmentation", "ionosphere"]
ks = [4,3,3,7,2]
kmeansTypes = ["random", "k-means++"]

In [6]:
for kmeansType in kmeansTypes:
    print ("--------- "+ kmeansType +" test ---------")
    
    for index, file in enumerate(files):
        print ("\n----- "+file+" -----\n")
        data = pd.read_csv("datasets/"+file+".data", header=None)
        print (data.shape)

        #display(data.head())
        if file == "segmentation": #Target eh na primeira coluna
            target = data.iloc[:,0]
            data = data.iloc[:,1:]
            
            
        else: #Target na última coluna
            target = data.iloc[:,-1]
            data = data.iloc[:,:-1]
        
        
        getWeight(data)
        
        clustering_times = []
        start = time()
        k = ks[index]
        kmeans = KMeans(n_clusters=k, random_state=100, init=kmeansType, n_init=1, max_iter=100).fit(data)
        #display(data)
        error = getSquaredError(data, kmeans)
        
        
                #print(kmeans.labels_)
        end = time()
        T1 = error/(data.shape[0])
        T2 = error/(data.shape[0]/2)
        
        #print(canopy(data.values, T1, T2))
        print("Erro quadrático médio: ",error)
        clustering_times.append(end - start)
        print(clustering_times)
        #print(kmeans.cluster_centers_)

--------- random test ---------

----- soybean-small -----

(47, 36)
Distância média:  5.186705273500234
Distância média:  7.892133380626508
[18.973665961010276, 17.233687939614086, 12.569805089976535, 11.874342087037917, 11.313708498984761, 11.704699910719626, 12.409673645990857, 13.564659966250536, 14.730919862656235, 15.362291495737216, 16.401219466856727, 17.029386365926403, 18.24828759089466, 18.973665961010276]
densidade:  [3] a :  [3.74048419] s:  18.973665961010276
densidade:  [4] a :  [2.03755056] s:  17.233687939614086
densidade:  [8] a :  [1.10563949] s:  12.569805089976535
densidade:  [10] a :  [0.53472532] s:  11.313708498984761
densidade:  [11] a :  [0.36438439] s:  11.704699910719626
densidade:  [12] a :  [0.34348455] s:  12.409673645990857
densidade:  [12] a :  [0.28237134] s:  13.564659966250536
densidade:  [10] a :  [0.05879447] s:  14.730919862656235
densidade:  [10] a :  [0.] s:  15.362291495737216
Erro quadrático médio:  100.72984563120583
[0.055190324783325195]

-

/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in true_divide


Distância média:  2.5437692122517013
Distância média:  57.83033498001055
[144.0915681086163, 143.09475881387132, 142.10087965948696, 141.09705879287492, 140.09553882975717, 138.1005068781429, 136.10742081165156, 132.10582878889184, 131.12501668255373, 130.1031898148543, 129.0939967620493, 128.10081966950875, 122.13058584973709, 112.11730464116589, 111.11719038924626, 109.13038073790452, 108.12090454671568, 106.14155642348572, 104.13035100296167, 103.14572216044638, 102.14670821911004, 99.13642115791754, 97.14525207131844, 95.1402648724503, 100.13955262532383, 102.13770116856949, 105.17675598724274, 107.14835509703357, 109.1457740821879, 117.16629208095647, 118.17880520634823, 120.10924194249166, 122.15801242652894, 125.11282907839627, 129.1006584026588, 130.11825390774348, 131.1379807683495, 135.1240541132481, 143.0966107215681, 144.0915681086163]
densidade:  [24] a :  [2.00925577] s:  144.0915681086163
densidade:  [24] a :  [1.92570681] s:  143.09475881387132
densidade:  [24] a :  [1.

/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in true_divide


Distância média:  17.468877956672003
Distância média:  68.91161529455735
[176.2863553993899, 169.11316595700052, 160.07627650592076, 156.26953893833564, 151.2868130406613, 143.60471475547033, 137.53392199744758, 122.17448956308351, 120.18015768004301, 117.98481724357588, 116.94567670504112, 111.6705292366791, 105.44824180611074, 99.65005519316082, 103.31222773708832, 106.10781026861312, 101.23060011676311, 103.52648115337448, 101.38225929619048, 96.57034379145598, 116.06026753372576, 127.10084500112498, 122.00379666223508, 132.1064460955634, 137.54163951327612, 130.2033528754156, 153.5590961161207, 150.38022110636757, 175.32674696120955, 176.2863553993899]
densidade:  [9] a :  [7.19325976] s:  176.2863553993899
densidade:  [9] a :  [5.58724207] s:  169.11316595700052
densidade:  [14] a :  [5.09118915] s:  160.07627650592076
densidade:  [14] a :  [4.63023055] s:  156.26953893833564
densidade:  [14] a :  [4.16408466] s:  151.2868130406613
densidade:  [15] a :  [3.7298365] s:  143.6047147

...

/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in true_divide


Distância média:  185.4874269607976
Distância média:  327.15025132762446
[1509.374242183851, 1514.8446950120867, 1508.2193553003722, 1510.8816733764595, 1505.565223781407, 1506.5441933839847, 1508.3060947870963, 1512.1785581094057, 1508.43493394224, 1513.4143778993973, 1508.1938505475205, 1506.5818989930565, 1511.4573275932878, 1506.317297157798, 1506.644466282014, 1506.36953357087, 1506.9026644861597, 1524.484180367125, 1227.5773829631382, 1405.560561622594, 1507.4531789837065, 1509.2443823090753, 1516.387827547706, 1524.484180367125, 1519.6826863479453, 1515.162743250754, 1507.946715788916, 1518.4867585853478, 1520.5263445992352, 1515.0725325103328, 1521.5626019892482]
densidade:  [19] a :  [17.98409739] s:  1509.374242183851
densidade:  [17] a :  [15.164162] s:  1514.8446950120867
densidade:  [19] a :  [16.70050697] s:  1508.2193553003722
densidade:  [23] a :  [16.956987] s:  1505.565223781407
densidade:  [17] a :  [5.56737167] s:  1508.3060947870963
densidade:  [17] a :  [4.9646987

/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in multiply
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in true_divide


Distância média:  3.9918803461829198
Distância média:  98.20031473947128
[303.0591963547983, 299.07372419624596, 293.0710071215101, 291.0651422661321, 289.0625560587469, 287.0632286713438, 285.0853207378258, 283.0708223046243, 282.08492860246224, 281.0822110259136, 279.09487259034694, 275.0815691775521, 273.0705347254006, 269.0592247470824, 267.08621199225206, 265.0388689611699, 259.0631271239043, 258.01411588918893, 257.0877758562427, 255.05218631796393, 253.08699470778066, 251.06165000959186, 247.04675064076758, 243.09903571492978, 242.0066906657851, 241.09850805265512, 239.07533665189348, 235.08901929953302, 233.09293373076795, 232.10121795239874, 231.07777797792716, 229.1038896117135, 212.0707087660269, 210.05777144889973, 174.06176815440946, 170.09287740471703, 155.15153882575578, 157.09994654080884, 158.08568715468203, 159.1298727651411, 160.08471693128422, 163.12265323982442, 175.10853777014987, 177.105359046207, 179.12007146045917, 180.1208491669965, 181.07180895987094, 182.076

/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in true_divide


In [7]:
#usando numpy em vez do pandas pode tornar mais rápido pra essas operações